In [1]:
import pandas as pd
import numpy as np
import os
import time
import pickle
from datetime import date, timedelta
from tqdm import tqdm
import re
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn import metrics
from MyTT import *
import warnings
def log_diff(df):
    df.replace(0, 0.01, inplace=True)
    diffdf = np.log(df).loc[1:,:] - np.log(df.shift(1)).loc[1:,:]
    return diffdf

def minicut(df, train_period=5, pred_period=1):
    '''
    train_period: 用于训练的数据长度，默认前5个交易日
    pred_period: 预测几日后的结果，默认第二天
    '''
    n_features = df.shape[1]
    roll = pd.concat([df.shift(i) for i in range(pred_period,pred_period+train_period-1)], axis=1)
    roll = pd.concat([df, roll], axis=1).dropna()
    return roll.iloc[-1:,:]

def cut(df, train_period=5, pred_period=1):
    '''
    train_period: 用于训练的数据长度，默认前5个交易日
    pred_period: 预测几日后的结果，默认第二天
    '''
    n_features = df.shape[1]
    roll = pd.concat([df.shift(i) for i in range(pred_period,pred_period+train_period)], axis=1)
    roll = pd.concat([df, roll], axis=1).dropna()
    train = roll.iloc[:, n_features:]
    pred = roll['Close'].iloc[:,0]
    return train, pred

In [2]:
data = pd.read_csv('EURUSD_M5.csv', names=['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'TickVol', 'Vol', 'Spread'], sep='\t').iloc[1:,2:].astype(float)
# data.iloc[:, 2:] = data.iloc[:, 2:].astype(float)
data

C:\Users\YLfan\AppData\Local\Temp\ipykernel_51476\4219125329.py:1: DtypeWarning: Columns (2,3,4,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('EURUSD_M5.csv', names=['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'TickVol', 'Vol', 'Spread'], sep='\t').iloc[1:,2:].astype(float)


,Open,High,Low,Close,TickVol,Vol,Spread
1,1.00730,1.00800,1.00730,1.00800,10.0,0.0,50.0
2,1.00790,1.00800,1.00760,1.00770,9.0,0.0,50.0
3,1.00780,1.00830,1.00760,1.00780,23.0,0.0,50.0
4,1.00790,1.00820,1.00780,1.00790,9.0,0.0,50.0
5,1.00820,1.00870,1.00800,1.00860,9.0,0.0,50.0
...,...,...,...,...,...,...,...
1813845,1.08510,1.08517,1.08496,1.08505,346.0,0.0,1.0
1813846,1.08505,1.08510,1.08484,1.08496,203.0,0.0,1.0
1813847,1.08496,1.08509,1.08486,1.08491,306.0,0.0,1.0
1813848,1.08491,1.08527,1.08490,1.08520,334.0,0.0,1.0


In [3]:
data['MA5'] = MA(data['Close'], 5)
data['MA15'] = MA(data['Close'], 15)
data['MA30'] = MA(data['Close'], 30)
data['MA60'] = MA(data['Close'], 60)
data['vMA5'] = MA(data['TickVol'], 5)
data['vMA15'] = MA(data['TickVol'], 15)
data['vMA30'] = MA(data['TickVol'], 30)
data['vMA60'] = MA(data['TickVol'], 60)
# dif, dea, macd = MACD(data['Close'])
# data['DIF'] = dif
# data['DEA'] = dea
# data['MACD'] = macd

In [4]:
data = data.dropna().drop(columns=['Vol', 'Spread']).reset_index(drop=True)
data

,Open,High,Low,Close,TickVol,MA5,MA15,MA30,MA60,vMA5,vMA15,vMA30,vMA60
0,1.01710,1.01710,1.01650,1.01660,3.0,1.016700,1.016993,1.017120,1.014390,5.2,12.533333,16.033333,13.916667
1,1.01670,1.01700,1.01670,1.01700,4.0,1.016740,1.016927,1.017197,1.014540,5.4,12.466667,15.933333,13.816667
2,1.01680,1.01700,1.01650,1.01650,9.0,1.016720,1.016860,1.017263,1.014687,6.8,11.866667,15.733333,13.816667
3,1.01710,1.01710,1.01640,1.01640,4.0,1.016640,1.016813,1.017317,1.014830,5.6,10.866667,15.266667,13.500000
4,1.01650,1.01710,1.01650,1.01710,3.0,1.016720,1.016793,1.017373,1.014983,4.6,9.200000,14.433333,13.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1813785,1.08510,1.08517,1.08496,1.08505,346.0,1.084976,1.084968,1.084775,1.084526,326.8,349.533333,460.300000,435.700000
1813786,1.08505,1.08510,1.08484,1.08496,203.0,1.084978,1.084985,1.084781,1.084544,294.6,330.266667,449.433333,434.700000
1813787,1.08496,1.08509,1.08486,1.08491,306.0,1.084982,1.085023,1.084786,1.084563,308.6,324.600000,443.866667,436.233333
1813788,1.08491,1.08527,1.08490,1.08520,334.0,1.085042,1.085067,1.084805,1.084593,304.6,323.200000,437.900000,437.150000


In [5]:
X, y = cut(log_diff(data), 10, 1)
X.columns = list(range(1,data.shape[1]*10+1))
X

,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,129,130
11,0.000590,0.000492,0.000295,0.000295,0.693147,0.000039,0.000013,-0.000046,0.000125,-0.389465,...,0.000393,0.287682,0.000039,-0.000066,0.000075,0.000148,0.037740,-0.005333,-0.006257,-0.007212
12,-0.000098,0.000000,-0.000197,0.000197,0.693147,0.000118,0.000020,-0.000029,0.000071,0.322773,...,-0.000492,0.810930,-0.000020,-0.000066,0.000066,0.000145,0.230524,-0.049325,-0.012632,0.000000
13,0.000491,0.000786,0.000197,0.000590,0.559616,0.000177,0.000072,-0.000007,0.000080,0.439367,...,-0.000098,-0.810930,-0.000079,-0.000046,0.000052,0.000141,-0.194156,-0.088033,-0.030110,-0.023186
14,-0.000098,0.000000,0.000590,0.000098,-1.435085,0.000216,0.000066,0.000003,0.000079,0.043485,...,0.000688,-0.287682,0.000079,-0.000020,0.000056,0.000151,-0.196710,-0.166497,-0.056131,-0.007435
15,0.000590,0.000098,0.000295,0.000196,0.000000,0.000275,0.000085,0.000000,0.000095,0.041673,...,-0.000393,1.673976,0.000020,-0.000026,-0.000013,0.000133,0.448025,-0.014599,-0.042459,0.008669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1813785,0.000009,0.000194,0.000147,0.000175,-0.058156,-0.000031,0.000036,0.000006,0.000016,0.070896,...,0.000074,-0.329815,0.000057,0.000035,0.000008,0.000025,-0.012230,-0.004568,-0.021570,0.005570
1813786,0.000175,0.000028,0.000055,-0.000037,0.035298,-0.000055,0.000025,-0.000002,0.000017,0.034873,...,0.000184,-0.202190,0.000085,0.000065,0.000010,0.000028,-0.095784,-0.017872,-0.015254,0.002140
1813787,-0.000046,-0.000065,-0.000111,-0.000083,-0.533233,0.000002,0.000016,0.000005,0.000017,-0.103730,...,-0.000065,0.239728,0.000138,0.000027,0.000004,0.000024,-0.005746,-0.114782,-0.016069,0.009026
1813788,-0.000083,-0.000009,0.000018,-0.000046,0.410379,0.000004,0.000035,0.000005,0.000018,0.046427,...,0.000249,-0.233061,0.000151,0.000037,0.000009,0.000024,-0.028695,-0.083765,-0.030147,-0.000039


In [6]:
y

11         0.000197
12         0.000590
13         0.000098
14         0.000196
15        -0.000393
             ...   
1813785   -0.000037
1813786   -0.000083
1813787   -0.000046
1813788    0.000267
1813789   -0.000037
Name: Close, Length: 1813779, dtype: float64

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
lgb.early_stopping(stopping_rounds=100, first_metric_only=True, verbose=0)
model = lgb.LGBMRegressor(boosting_type='gbdt', device='gpu', learning_rate=0.01, metric='l1', n_estimators=4096, num_leaves=32768, min_data_in_leaf=128, bagging_fraction=0.8, bagging_freq=10, verbose=0 ,num_iterations=10000, early_stopping_round=100, first_metric_only=True,n_jobs=-1)

model.fit(X_train,y_train,eval_metric='l2',eval_set=[(X_test,y_test)])

c:\Users\YLfan\miniconda3\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=128, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=128
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] min_data_in_leaf is set=128, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=128
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] min_data_in_leaf is set=128, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=128
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=10, subsample_fr

LGBMRegressor(bagging_fraction=0.8, bagging_freq=10, device='gpu',
              early_stopping_round=100, first_metric_only=True,
              learning_rate=0.01, metric='l1', min_data_in_leaf=128,
              n_estimators=4096, n_jobs=-1, num_iterations=10000,
              num_leaves=32768, verbose=0)

In [8]:
filename = 'model/{}.pickle'.format('EURUSD_M5_10_1')
pickle.dump(model, open(filename,'wb'))

In [9]:
#model = pickle.load(open('model/EURUSD1.pickle', 'rb'))
model.best_score_

defaultdict(collections.OrderedDict,
            {'valid_0': OrderedDict([('l2', 1.374820453913781e-07),
                          ('l1', 0.00023367515496807776)])})

In [10]:
model.predict(X[-1000:])

[LightGBM] [Warning] min_data_in_leaf is set=128, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=128
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10


array([-5.31715960e-06,  1.50254597e-06,  2.45722369e-06,  1.41906112e-07,
       -9.60381257e-08,  8.51830737e-06, -1.59073321e-05, -4.02678629e-06,
       -4.94166876e-06, -3.33132664e-06, -7.59318872e-07, -5.95046554e-06,
        5.53421387e-06,  2.35927959e-06,  3.45530329e-06,  5.84472051e-06,
       -1.09212542e-05,  1.19413305e-05, -1.13528775e-05, -8.21211345e-06,
        1.21485967e-05,  4.96255265e-06, -1.35547228e-05,  1.17694074e-05,
        2.35436928e-05,  1.02690793e-05,  9.84170979e-06,  2.92260126e-06,
       -4.21530609e-06,  5.22847023e-07, -6.97236007e-06,  1.10107306e-06,
        5.86673451e-06, -9.17140523e-07, -7.01112936e-06,  2.03489426e-06,
        1.76304076e-05, -1.60417339e-05,  7.45726116e-07,  1.28564611e-05,
        1.03300850e-05,  1.78955250e-05, -2.47350281e-05, -5.31898993e-06,
       -5.86344041e-06,  8.51068976e-06, -7.56468040e-06, -1.65291251e-05,
        3.63981985e-06,  1.95991983e-06,  1.01403467e-05, -1.05029644e-05,
        1.16270838e-05, -

In [16]:
pchg = (np.exp(1.45670666e-05)-1)*3.3/1e-5
1.088*(1+2*pchg*1e-5)

1.0881046039537265

In [16]:
data = pd.read_csv('EURUSD_M5.csv', names=['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Vol'], header=None)
# data.iloc[:, 2:] = data.iloc[:, 2:].astype(float)
data = data.iloc[-30000:, :]
data

,Date,Time,Open,High,Low,Close,Vol
1783850,2023.12.28\t14:10:00\t1.10850\t1.10901\t1.1084...,NaN,NaN,NaN,NaN,NaN,NaN
1783851,2023.12.28\t14:15:00\t1.10885\t1.10954\t1.1088...,NaN,NaN,NaN,NaN,NaN,NaN
1783852,2023.12.28\t14:20:00\t1.10947\t1.10967\t1.1092...,NaN,NaN,NaN,NaN,NaN,NaN
1783853,2023.12.28\t14:25:00\t1.10951\t1.10960\t1.1088...,NaN,NaN,NaN,NaN,NaN,NaN
1783854,2023.12.28\t14:30:00\t1.10902\t1.10949\t1.1088...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1813845,2024.05.24\t15:55:00\t1.08510\t1.08517\t1.0849...,NaN,NaN,NaN,NaN,NaN,NaN
1813846,2024.05.24\t16:00:00\t1.08505\t1.08510\t1.0848...,NaN,NaN,NaN,NaN,NaN,NaN
1813847,2024.05.24\t16:05:00\t1.08496\t1.08509\t1.0848...,NaN,NaN,NaN,NaN,NaN,NaN
1813848,2024.05.24\t16:10:00\t1.08491\t1.08527\t1.0849...,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
model = pickle.load(open('model/EURUSD1.pickle', 'rb')).booster_
X, y = cut(log_diff(data.drop(columns=['Date', 'Time'])), 10, 1)
X.columns = list(range(1,51))
error = y - model.predict(X)
error

8902358    0.000061
8902359   -0.000100
8902360   -0.000052
8902361   -0.000097
8902362   -0.000011
             ...   
8932342    0.000149
8932343    0.000090
8932344    0.000053
8932345   -0.000094
8932346    0.000018
Name: Close, Length: 29989, dtype: float64

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, error, test_size=0.2)
lgb.early_stopping(stopping_rounds=100, first_metric_only=True, verbose=0)
minimodel = lgb.LGBMRegressor(boosting_type='rf', device='gpu', learning_rate=0.001, metric='l1', n_estimators=10000, num_leaves=256, min_data_in_leaf=32, bagging_fraction=0.8, bagging_freq=3, verbose=0 ,num_iterations=1000000, early_stopping_round=100, first_metric_only=True,n_jobs=-1)

minimodel.fit(X_train,y_train,eval_metric='l2',eval_set=[(X_test,y_test)])

c:\Users\YLfan\miniconda3\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] min_data_in_leaf is set=32, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=32
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] min_data_in_leaf is set=32, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=32
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] min_data_in_leaf is set=32, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=32
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will b

LGBMRegressor(bagging_fraction=0.8, bagging_freq=3, boosting_type='rf',
              device='gpu', early_stopping_round=100, first_metric_only=True,
              learning_rate=0.001, metric='l1', min_data_in_leaf=32,
              n_estimators=10000, n_jobs=-1, num_iterations=1000000,
              num_leaves=256, verbose=0)

In [25]:
filename = 'model/mini{}.pickle'.format('EURUSD1')
pickle.dump(minimodel, open(filename,'wb'))